### 1. mongodb 문법 실습
* cine 21 랭킹사이트 크롤링
    - 사이즈 주소 : http://www.cine21.com/rank/person
    - 요청 방식 확인 방법 : 개발자 모드 들어가서(Command + option + i) Network -> content 페이지 확인
    - Request URL : http://www.cine21.com/rank/person/content
    - Request Method : POST
    - Form data
        - section = 'actor'
        - period_start = '2021-12'
        - gender = 'all'
        - page = 3

#### 배우 랭킹 및 상세 정보 크롤링 및 mongodb 저장 (full source)

In [6]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re                   

connection = pymongo.MongoClient()
actor_db = connection.cine21
actor_collection = actor_db.actor_collection

actors_info_list = []

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = {}
post_data['section'] = 'actor'
post_data['period_start'] = '2021-01'
post_data['gender'] = 'all'

for index in range(1, 21):

    post_data['page'] = index

    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('ul.people_list > li.people_li > span.grade')
    
    for index, actor in enumerate(actors):

        # 전체 배우의 기본 정보 크롤링
        actor_name = re.sub('\(\w*\)','',actor.text)
        actor_hits = int(hits[index].text.replace(',', ''))
        movie_titles = movies[index].select('li a span')

        movie_title_list = []

        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        movie_title_list

        actor_info_dict = {}
        actor_info_dict["배우이름"] = actor_name
        actor_info_dict["흥행지수"] = actor_hits
        actor_info_dict["출연영화"] = movie_title_list
        actor_info_dict["랭킹"] = rankings[index].text

        # 각 배우별 링크로 이동 후 자세한 정보 데이터 크롤링
        actor_link = 'http://www.cine21.com/' + actor.select_one('a').attrs['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')

        # 각 배우별 자세한 정보 리스트를 각 각 크롤링
        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text
            actor_item_value = re.sub('<span.*?>.*?</span>','',str(actor_item))
            actor_item_value = re.sub('<.*?>','', actor_item_value)
            actor_info_dict[actor_item_field] = actor_item_value

        actors_info_list.append(actor_info_dict)

actor_collection.insert_many(actors_info_list)

In [7]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re                   

connection = pymongo.MongoClient()
actor_db = connection.cine21
actor_collection = actor_db.actor_collection

actor_collection.find_one({})

{'_id': ObjectId('61d937e1f92a9ab995022821'),
 '배우이름': '김윤석',
 '흥행지수': 46285,
 '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987'],
 '랭킹': '1',
 '직업': '배우',
 '생년월일': '1968-01-21',
 '성별': '남',
 '신장/체중': '178cm, 68kg',
 '학교': '동의대학교 학사',
 '취미': '여행, 낚시',
 '소속사': '심엔터테인먼트'}

In [8]:
docs = actor_collection.find({}).limit(3)

for doc in docs:
    print(doc)

{'_id': ObjectId('61d937e1f92a9ab995022821'), '배우이름': '김윤석', '흥행지수': 46285, '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987'], '랭킹': '1', '직업': '배우', '생년월일': '1968-01-21', '성별': '남', '신장/체중': '178cm, 68kg', '학교': '동의대학교 학사', '취미': '여행, 낚시', '소속사': '심엔터테인먼트'}
{'_id': ObjectId('61d937e1f92a9ab995022822'), '배우이름': '조인성', '흥행지수': 40500, '출연영화': ['모가디슈', '클래식'], '랭킹': '2', '다른 이름': '趙寅成', '직업': '배우', '생년월일': '1981-07-28', '성별': '남', '신장/체중': '186cm, 72kg', '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)', '취미': '모자모으기, 영화보기, 농구', '특기': '농구, 태권도(공인 4단)', '소속사': '싸이더스 HQ'}
{'_id': ObjectId('61d937e1f92a9ab995022823'), '배우이름': '허준호', '흥행지수': 34683, '출연영화': ['모가디슈', '국가부도의 날'], '랭킹': '3', '원어명': '許俊豪', '직업': '배우', '생년월일': '1964-03-03', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과', '취미': '만화책 보기', '특기': '야구, 농구', '소속사': '지티비엔터테인먼트'}


#### 1.1 컬럼명 변경
* 저장되어 있는 mongodb 데이터의 컬럼명을 변경하는 방법
* update_one()/update_many() 함수 활용

In [9]:
actor_collection.update_many( {}, {'$rename':{'다른 이름':'다른이름'}})

In [10]:
docs = actor_collection.find({}).limit(3)
for doc in docs:
    print(doc)

{'_id': ObjectId('61d937e1f92a9ab995022821'), '배우이름': '김윤석', '흥행지수': 46285, '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987'], '랭킹': '1', '직업': '배우', '생년월일': '1968-01-21', '성별': '남', '신장/체중': '178cm, 68kg', '학교': '동의대학교 학사', '취미': '여행, 낚시', '소속사': '심엔터테인먼트'}
{'_id': ObjectId('61d937e1f92a9ab995022822'), '배우이름': '조인성', '흥행지수': 40500, '출연영화': ['모가디슈', '클래식'], '랭킹': '2', '직업': '배우', '생년월일': '1981-07-28', '성별': '남', '신장/체중': '186cm, 72kg', '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)', '취미': '모자모으기, 영화보기, 농구', '특기': '농구, 태권도(공인 4단)', '소속사': '싸이더스 HQ', '다른이름': '趙寅成'}
{'_id': ObjectId('61d937e1f92a9ab995022823'), '배우이름': '허준호', '흥행지수': 34683, '출연영화': ['모가디슈', '국가부도의 날'], '랭킹': '3', '원어명': '許俊豪', '직업': '배우', '생년월일': '1964-03-03', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과', '취미': '만화책 보기', '특기': '야구, 농구', '소속사': '지티비엔터테인먼트'}


#### 참고: collection 변수명 변경

In [11]:
actor = actor_collection

#### 1.2 find의 다양한 문법 - sort

In [13]:
## '생년월일' sort 시, '생년월일' 데이터가 없는 순서부터 정렬
## pymongo.DESCENDING 으로 내림차순 정렬

docs = actor.find({}).sort('생년월일', pymongo.DESCENDING).limit(10)

for doc in docs:
    print(doc)

{'_id': ObjectId('61d937e1f92a9ab995022871'), '배우이름': '김강훈', '흥행지수': 1984, '출연영화': ['내일의 기억'], '랭킹': '81', '직업': '배우', '생년월일': '2009-06-07', '성별': '남', '홈페이지': '\nhttp://instagram.com/kanghoon.juwon\n'}
{'_id': ObjectId('61d937e1f92a9ab995022849'), '배우이름': '이재인', '흥행지수': 7507, '출연영화': ['발신제한', '아이 캔 스피크', '봉오동 전투', '어른도감'], '랭킹': '41', '직업': '배우', '생년월일': '2004-02-06', '성별': '여'}
{'_id': ObjectId('61d937e1f92a9ab995022844'), '배우이름': '남다름', '흥행지수': 8151, '출연영화': ['싱크홀', '제8일의 밤'], '랭킹': '36', '직업': '배우', '생년월일': '2002-06-13', '성별': '남'}
{'_id': ObjectId('61d937e1f92a9ab995022898'), '배우이름': '김향기', '흥행지수': 1182, '출연영화': ['아이', '신과 함께-죄와 벌', '신과 함께-인과 연', '늑대소년', '증인'], '랭킹': '120', '직업': '배우', '생년월일': '2000-08-09', '성별': '여', '홈페이지': '\nhttps://twitter.com/smell2001\n', '특기': '애교 부리기, 피아노 치기'}
{'_id': ObjectId('61d937e1f92a9ab99502285f'), '배우이름': '성유빈', '흥행지수': 3520, '출연영화': ['장르만 로맨스', '윤희에게', '아이 캔 스피크', '봉오동 전투'], '랭킹': '63', '직업': '배우', '생년월일': '2000-07-25', '성별': '남', '홈페이지': '\nhttp

#### 1.3 find의 다양한 문법 - exists

In [19]:
docs = actor.find({"특기": {"$exists" : True} }).sort('흥행지수').limit(5)

for doc in docs:
    print(doc)

{'_id': ObjectId('61d937e1f92a9ab9950228a7'), '배우이름': '박근형', '흥행지수': 965, '출연영화': ['아들의 이름으로', '동백', '장수상회', '증인'], '랭킹': '135', '직업': '배우', '생년월일': '1940-06-07', '성별': '남', '신장/체중': '177cm, 80kg', '학교': '중앙대학교 연극영화학', '취미': '낚시, 골프', '특기': '수영'}
{'_id': ObjectId('61d937e1f92a9ab995022899'), '배우이름': '이연희', '흥행지수': 1176, '출연영화': ['새해전야'], '랭킹': '121', '직업': '배우', '생년월일': '1988-01-09', '성별': '여', '홈페이지': '\nhttps://www.facebook.com/leeyeonhee\n', '신장/체중': '170cm, 46kg', '학교': '중앙대학교 연극영화학', '취미': '영화감상, 독서', '특기': '수영', '소속사': 'SM엔터테인먼트'}
{'_id': ObjectId('61d937e1f92a9ab995022898'), '배우이름': '김향기', '흥행지수': 1182, '출연영화': ['아이', '신과 함께-죄와 벌', '신과 함께-인과 연', '늑대소년', '증인'], '랭킹': '120', '직업': '배우', '생년월일': '2000-08-09', '성별': '여', '홈페이지': '\nhttps://twitter.com/smell2001\n', '특기': '애교 부리기, 피아노 치기'}
{'_id': ObjectId('61d937e1f92a9ab995022895'), '배우이름': '이동욱', '흥행지수': 1278, '출연영화': ['해피 뉴 이어', '뷰티 인사이드', '아랑'], '랭킹': '117', '직업': '배우', '생년월일': '1981-11-06', '성별': '남', '홈페이지': '\nhttps://www.fac

In [22]:
## 상세 정보란에 특기가 없는 배우의 이름을 5명을 추출하기

docs = actor.find({"특기": {"$exists" : False }}, {'배우이름':1, '_id':0} ).limit(5)

for doc in docs:
    print(doc)

{'배우이름': '김윤석'}
{'배우이름': '김소진'}
{'배우이름': '조우진'}
{'배우이름': '정만식'}
{'배우이름': '김재범'}


#### 1.4 find의 다양한 문법 - 필드값 범위로 검색

In [24]:
docs = actor.find({'흥행지수' : {'$gte' : 10000 }})

for doc in docs:
    print(doc)

{'_id': ObjectId('61d937e1f92a9ab995022821'), '배우이름': '김윤석', '흥행지수': 46285, '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987'], '랭킹': '1', '직업': '배우', '생년월일': '1968-01-21', '성별': '남', '신장/체중': '178cm, 68kg', '학교': '동의대학교 학사', '취미': '여행, 낚시', '소속사': '심엔터테인먼트'}
{'_id': ObjectId('61d937e1f92a9ab995022822'), '배우이름': '조인성', '흥행지수': 40500, '출연영화': ['모가디슈', '클래식'], '랭킹': '2', '직업': '배우', '생년월일': '1981-07-28', '성별': '남', '신장/체중': '186cm, 72kg', '학교': '전남과학대 모델이벤트과 - 동국대 연극영화 (중퇴)', '취미': '모자모으기, 영화보기, 농구', '특기': '농구, 태권도(공인 4단)', '소속사': '싸이더스 HQ', '다른이름': '趙寅成'}
{'_id': ObjectId('61d937e1f92a9ab995022823'), '배우이름': '허준호', '흥행지수': 34683, '출연영화': ['모가디슈', '국가부도의 날'], '랭킹': '3', '원어명': '許俊豪', '직업': '배우', '생년월일': '1964-03-03', '성별': '남', '신장/체중': '180cm, 75kg', '학교': '서울예술대학 연극과', '취미': '만화책 보기', '특기': '야구, 농구', '소속사': '지티비엔터테인먼트'}
{'_id': ObjectId('61d937e1f92a9ab995022824'), '배우이름': '김소진', '흥행지수': 28917, '출연영화': ['모가디슈', '스프링 송', '아이 캔 스피크', '미성년'], '랭킹': '4', '직업': '배우', '성

In [25]:
docs = actor.find({'흥행지수' : {'$lte' : 1000 }})

for doc in docs:
    print(doc)

{'_id': ObjectId('61d937e1f92a9ab9950228a7'), '배우이름': '박근형', '흥행지수': 965, '출연영화': ['아들의 이름으로', '동백', '장수상회', '증인'], '랭킹': '135', '직업': '배우', '생년월일': '1940-06-07', '성별': '남', '신장/체중': '177cm, 80kg', '학교': '중앙대학교 연극영화학', '취미': '낚시, 골프', '특기': '수영'}
{'_id': ObjectId('61d937e1f92a9ab9950228a8'), '배우이름': '김준배', '흥행지수': 930, '출연영화': ['강릉', '악인은 너무 많다 2: 제주 실종사건의 전말', '부러진 화살', '바람의 언덕', '잉투기'], '랭킹': '136', '직업': '배우', '생년월일': '1969-07-16', '성별': '남', '다른이름': '김태한'}
{'_id': ObjectId('61d937e1f92a9ab9950228a9'), '배우이름': '태항호', '흥행지수': 924, '출연영화': ['파이프라인', '이 안에 외계인이 있다'], '랭킹': '137', '직업': '배우', '생년월일': '1983-07-22', '성별': '남'}
{'_id': ObjectId('61d937e1f92a9ab9950228aa'), '배우이름': '정진영', '흥행지수': 918, '출연영화': ['해피 뉴 이어', '판도라', '초록물고기'], '랭킹': '138', '원어명': '정진영', '직업': '배우', '생년월일': '1964-10-16', '성별': '남', '학교': '서울대학교 국어국문학'}
{'_id': ObjectId('61d937e1f92a9ab9950228ab'), '배우이름': '조현정', '흥행지수': 908, '출연영화': ['극장판 도라에몽: 진구의 신공룡', '도라에몽: 스탠바이미2', '신비아파트: 금빛 도깨비와 비밀의 동굴', '기기괴괴 성형수'], '랭킹': 

In [39]:
docs = actor.find({'흥행지수':{'$gte' : 10000}, '출연영화':'모가디슈' }, {'배우이름':1, '출연영화':1, '_id':0} ).sort('배우이름')

for doc in docs:
    print(doc)

{'배우이름': '구교환', '출연영화': ['모가디슈', '메기', '우리 손자 베스트', '반도', '오늘영화']}
{'배우이름': '김소진', '출연영화': ['모가디슈', '스프링 송', '아이 캔 스피크', '미성년']}
{'배우이름': '김윤석', '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987']}
{'배우이름': '김재화', '출연영화': ['싱크홀', '모가디슈', '괴기맨숀', '액션히어로', '싸나희 순정', '숏버스 이별행']}
{'배우이름': '정만식', '출연영화': ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '똥파리', '끝까지 간다']}
{'배우이름': '조인성', '출연영화': ['모가디슈', '클래식']}
{'배우이름': '허준호', '출연영화': ['모가디슈', '국가부도의 날']}


#### 1.5 find의 다양한 문법 - or

In [41]:
docs = actor.find({'$or': [{'출연영화':'모가디슈'}, {'출연영화':'타짜'}]}, {'배우이름':1, '출연영화':1, '_id':0} ).sort('배우이름')

for doc in docs:
    print(doc)

{'배우이름': '구교환', '출연영화': ['모가디슈', '메기', '우리 손자 베스트', '반도', '오늘영화']}
{'배우이름': '김소진', '출연영화': ['모가디슈', '스프링 송', '아이 캔 스피크', '미성년']}
{'배우이름': '김윤석', '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987']}
{'배우이름': '김재화', '출연영화': ['싱크홀', '모가디슈', '괴기맨숀', '액션히어로', '싸나희 순정', '숏버스 이별행']}
{'배우이름': '박경혜', '출연영화': ['모가디슈', '메기', '1987']}
{'배우이름': '정만식', '출연영화': ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '똥파리', '끝까지 간다']}
{'배우이름': '조인성', '출연영화': ['모가디슈', '클래식']}
{'배우이름': '허준호', '출연영화': ['모가디슈', '국가부도의 날']}


In [47]:
docs = actor.find(
                    {'흥행지수' : {'$gte':10000 }, '$or': [{'출연영화':'모가디슈'}, {'출연영화':'타짜'}]},
                    {'배우이름':1, '출연영화':1, '_id':0}
                 ).sort('배우이름')

for doc in docs:
    print(doc)

{'배우이름': '구교환', '출연영화': ['모가디슈', '메기', '우리 손자 베스트', '반도', '오늘영화']}
{'배우이름': '김소진', '출연영화': ['모가디슈', '스프링 송', '아이 캔 스피크', '미성년']}
{'배우이름': '김윤석', '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987']}
{'배우이름': '김재화', '출연영화': ['싱크홀', '모가디슈', '괴기맨숀', '액션히어로', '싸나희 순정', '숏버스 이별행']}
{'배우이름': '정만식', '출연영화': ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '똥파리', '끝까지 간다']}
{'배우이름': '조인성', '출연영화': ['모가디슈', '클래식']}
{'배우이름': '허준호', '출연영화': ['모가디슈', '국가부도의 날']}


#### find의 다양한 문법 - nor
* not or

In [54]:
docs = actor.find({'$or': [{'흥행지수':{'$gte':10000}}, {'흥행지수':{'$lte':2000}}]},{'배우이름':1, '흥행지수':1, '_id':0})

for doc in docs:
    print(doc)

{'배우이름': '김윤석', '흥행지수': 46285}
{'배우이름': '조인성', '흥행지수': 40500}
{'배우이름': '허준호', '흥행지수': 34683}
{'배우이름': '김소진', '흥행지수': 28917}
{'배우이름': '황정민', '흥행지수': 24566}
{'배우이름': '조우진', '흥행지수': 23907}
{'배우이름': '정만식', '흥행지수': 23125}
{'배우이름': '차승원', '흥행지수': 22415}
{'배우이름': '김재범', '흥행지수': 21049}
{'배우이름': '김성균', '흥행지수': 20366}
{'배우이름': '김재화', '흥행지수': 19796}
{'배우이름': '한예리', '흥행지수': 19681}
{'배우이름': '이광수', '흥행지수': 18967}
{'배우이름': '이유미', '흥행지수': 18186}
{'배우이름': '김영광', '흥행지수': 17675}
{'배우이름': '박정민', '흥행지수': 17461}
{'배우이름': '구교환', '흥행지수': 17365}
{'배우이름': '김홍파', '흥행지수': 16594}
{'배우이름': '윤여정', '흥행지수': 16547}
{'배우이름': '지창욱', '흥행지수': 15718}
{'배우이름': '고창석', '흥행지수': 14256}
{'배우이름': '류경수', '흥행지수': 14033}
{'배우이름': '이성민', '흥행지수': 13088}
{'배우이름': '강하늘', '흥행지수': 12425}
{'배우이름': '권소현', '흥행지수': 12221}
{'배우이름': '진경', '흥행지수': 11262}
{'배우이름': '윤계상', '흥행지수': 10590}
{'배우이름': '정재원', '흥행지수': 10524}
{'배우이름': '김혜준', '흥행지수': 10184}
{'배우이름': '김강훈', '흥행지수': 1984}
{'배우이름': '오대환', '흥행지수': 1979}
{'배우이름': '진선규', '흥행지수': 1916}
{'배우이름': '노회

In [56]:
docs = actor.find({'$nor': [{'흥행지수':{'$gte':10000}}, {'흥행지수':{'$lte':2000}}]},{'배우이름':1, '흥행지수':1, '_id':0})

for doc in docs:
    print(doc)

{'배우이름': '임윤아', '흥행지수': 9812}
{'배우이름': '박용우', '흥행지수': 9265}
{'배우이름': '손석구', '흥행지수': 8980}
{'배우이름': '김강우', '흥행지수': 8815}
{'배우이름': '배유람', '흥행지수': 8715}
{'배우이름': '이선빈', '흥행지수': 8426}
{'배우이름': '남다름', '흥행지수': 8151}
{'배우이름': '임지연', '흥행지수': 7941}
{'배우이름': '공유', '흥행지수': 7809}
{'배우이름': '공민정', '흥행지수': 7777}
{'배우이름': '염혜란', '흥행지수': 7715}
{'배우이름': '이재인', '흥행지수': 7507}
{'배우이름': '천우희', '흥행지수': 7496}
{'배우이름': '이규원', '흥행지수': 7016}
{'배우이름': '박지환', '흥행지수': 6619}
{'배우이름': '김슬기', '흥행지수': 6415}
{'배우이름': '유승목', '흥행지수': 6346}
{'배우이름': '류승룡', '흥행지수': 6328}
{'배우이름': '박보검', '흥행지수': 6247}
{'배우이름': '이학주', '흥행지수': 6109}
{'배우이름': '박경혜', '흥행지수': 5783}
{'배우이름': '오나라', '흥행지수': 5625}
{'배우이름': '설경구', '흥행지수': 5499}
{'배우이름': '서예지', '흥행지수': 5387}
{'배우이름': '김희원', '흥행지수': 4986}
{'배우이름': '변요한', '흥행지수': 4531}
{'배우이름': '이수경', '흥행지수': 4359}
{'배우이름': '이유영', '흥행지수': 4344}
{'배우이름': '이성욱', '흥행지수': 3973}
{'배우이름': '장영남', '흥행지수': 3836}
{'배우이름': '김지호', '흥행지수': 3756}
{'배우이름': '강소라', '흥행지수': 3745}
{'배우이름': '이정은', '흥행지수': 3635}
{'배우이름': '성

#### find의 다양한 문법 - in, nin
* in
* not in

In [62]:
docs = actor.find({'흥행지수': {'$in': [2338, 2435]}}, {'배우이름':1, '흥행지수':1, '_id':0})

for doc in docs:
    print(doc)

{'배우이름': '한지민', '흥행지수': 2435}
{'배우이름': '장혁', '흥행지수': 2338}


In [63]:
docs = actor.find({'흥행지수': {'$nin': [2338, 2435]}}, {'배우이름':1, '흥행지수':1, '_id':0}).limit(3)

for doc in docs:
    print(doc)

{'배우이름': '김윤석', '흥행지수': 46285}
{'배우이름': '조인성', '흥행지수': 40500}
{'배우이름': '허준호', '흥행지수': 34683}


In [72]:
## 흥행지수가 2338, 2435가 아니고, 10000 이하인 데이터 3개를 추출 (nor, in, gt 활용, 배우이름과 흥행지수만 출력)

### 방법 1

docs = actor.find(
{'$nor': [{'흥행지수':{'$in':[2338, 2435]}}, {'흥행지수':{'$gt':10000}}]},{'배우이름':1, '흥행지수':1, '_id':0}).limit(3)

for doc in docs:
    print(doc)

{'배우이름': '임윤아', '흥행지수': 9812}
{'배우이름': '박용우', '흥행지수': 9265}
{'배우이름': '손석구', '흥행지수': 8980}


In [79]:
## 흥행지수가 2338, 2435가 아니고, 10000 이하인 데이터 3개를 추출 (nor, in, gt 활용, 배우이름과 흥행지수만 출력)

### 방법 2

docs = actor.find(
{'흥행지수' : {'$nin':[2338,2435]}, '흥행지수': {'$lte':10000}}, {'배우이름':1, '흥행지수':1, '_id':0}).limit(3)

for doc in docs:
    print(doc)

{'배우이름': '임윤아', '흥행지수': 9812}
{'배우이름': '박용우', '흥행지수': 9265}
{'배우이름': '손석구', '흥행지수': 8980}


#### find의 다양한 문법 - skip, limit
* skip(n): 검색 결과 n개 만큼 건너뜀
* limit(n): 검색 결과 n개만 표시

In [87]:
docs = actor.find({'흥행지수': {'$gte':20000}}, {'흥행지수':1, '배우이름':1, '_id':0}).limit(3)

for doc in docs:
    print(doc)

{'배우이름': '김윤석', '흥행지수': 46285}
{'배우이름': '조인성', '흥행지수': 40500}
{'배우이름': '허준호', '흥행지수': 34683}


In [88]:
docs = actor.find({'흥행지수': {'$gte':20000}}, {'흥행지수':1, '배우이름':1, '_id':0}).skip(3)

for doc in docs:
    print(doc)

{'배우이름': '김소진', '흥행지수': 28917}
{'배우이름': '황정민', '흥행지수': 24566}
{'배우이름': '조우진', '흥행지수': 23907}
{'배우이름': '정만식', '흥행지수': 23125}
{'배우이름': '차승원', '흥행지수': 22415}
{'배우이름': '김재범', '흥행지수': 21049}
{'배우이름': '김성균', '흥행지수': 20366}


In [89]:
docs = actor.find({'흥행지수': {'$gte':20000}}, {'흥행지수':1, '배우이름':1, '_id':0}).skip(3).limit(3)

for doc in docs:
    print(doc)

{'배우이름': '김소진', '흥행지수': 28917}
{'배우이름': '황정민', '흥행지수': 24566}
{'배우이름': '조우진', '흥행지수': 23907}


#### find의 다양한 문법 - list 검색

In [93]:
docs = actor.find({'출연영화':'모가디슈'}, {'배우이름':1, '출연영화':1, '_id':0})

for doc in docs:
    print(doc)

{'배우이름': '김윤석', '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987']}
{'배우이름': '조인성', '출연영화': ['모가디슈', '클래식']}
{'배우이름': '허준호', '출연영화': ['모가디슈', '국가부도의 날']}
{'배우이름': '김소진', '출연영화': ['모가디슈', '스프링 송', '아이 캔 스피크', '미성년']}
{'배우이름': '정만식', '출연영화': ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '똥파리', '끝까지 간다']}
{'배우이름': '김재화', '출연영화': ['싱크홀', '모가디슈', '괴기맨숀', '액션히어로', '싸나희 순정', '숏버스 이별행']}
{'배우이름': '구교환', '출연영화': ['모가디슈', '메기', '우리 손자 베스트', '반도', '오늘영화']}
{'배우이름': '박경혜', '출연영화': ['모가디슈', '메기', '1987']}


In [95]:
## 출연영화 = ['모가디슈', '메기'] 해당되는 document가 없기 때문에 출력값이 없음

docs = actor.find({'출연영화':['모가디슈', '메기']}, {'배우이름':1, '출연영화':1, '_id':0})

for doc in docs:
    print(doc)

#### find의 다양한 문법 - list 검색 (all)

In [99]:
## $all : 리스트 안에 해당 데이터가 있으면 출력

docs = actor.find({'출연영화': {'$all':['모가디슈', '메기']}}, {'배우이름':1, '출연영화':1, '_id':0})

for doc in docs:
    print(doc)

{'배우이름': '구교환', '출연영화': ['모가디슈', '메기', '우리 손자 베스트', '반도', '오늘영화']}
{'배우이름': '박경혜', '출연영화': ['모가디슈', '메기', '1987']}


#### find의 다양한 문법 - list 검색 (리스트 인덱스 번호로 검색)

In [105]:
## 출연영화[0] = '모가디슈' 만족하는 document 출력

docs = actor.find({'출연영화.0':'모가디슈'}, {'배우이름':1, '출연영화':1, '_id':0})

for doc in docs:
    print(doc)

{'배우이름': '김윤석', '출연영화': ['모가디슈', '타짜', '당신, 거기 있어줄래요', '화이 : 괴물을 삼킨 아이', '미성년', '1987']}
{'배우이름': '조인성', '출연영화': ['모가디슈', '클래식']}
{'배우이름': '허준호', '출연영화': ['모가디슈', '국가부도의 날']}
{'배우이름': '김소진', '출연영화': ['모가디슈', '스프링 송', '아이 캔 스피크', '미성년']}
{'배우이름': '정만식', '출연영화': ['모가디슈', '지푸라기라도 잡고 싶은 짐승들', '똥파리', '끝까지 간다']}
{'배우이름': '구교환', '출연영화': ['모가디슈', '메기', '우리 손자 베스트', '반도', '오늘영화']}
{'배우이름': '박경혜', '출연영화': ['모가디슈', '메기', '1987']}


In [106]:
## 출연영화[1] = '모가디슈' 만족하는 document 출력

docs = actor.find({'출연영화.1':'모가디슈'}, {'배우이름':1, '출연영화':1, '_id':0})

for doc in docs:
    print(doc)

{'배우이름': '김재화', '출연영화': ['싱크홀', '모가디슈', '괴기맨숀', '액션히어로', '싸나희 순정', '숏버스 이별행']}


#### find의 다양한 문법 - list 검색 (리스트 사이즈로 검색)

In [107]:
## len(출연영화) = 5 만족하는 document 출력

docs = actor.find({'출연영화': {'$size':5}}, {'배우이름':1, '출연영화':1, '_id':0})

for doc in docs:
    print(doc)

{'배우이름': '황정민', '출연영화': ['인질', '와이키키 브라더스', '바람난 가족', '신세계', '내 생애 가장 아름다운 일주일']}
{'배우이름': '한예리', '출연영화': ['미나리', '최악의 하루', '더 테이블', '춘몽', '백년해로외전']}
{'배우이름': '구교환', '출연영화': ['모가디슈', '메기', '우리 손자 베스트', '반도', '오늘영화']}
{'배우이름': '김강우', '출연영화': ['내일의 기억', '새해전야', '귀문', '돈의 맛', '하하하']}
{'배우이름': '배유람', '출연영화': ['내일의 기억', '파이프라인', '연애 빠진 로맨스', '찬실이는 복도 많지', '소셜포비아']}
{'배우이름': '천우희', '출연영화': ['비와 당신의 이야기', '메기', '뷰티 인사이드', '버티고', '감쪽같은 그녀']}
{'배우이름': '설경구', '출연영화': ['킹메이커', '자산어보', '불한당: 나쁜 놈들의 세상', '오아시스', '1987']}
{'배우이름': '김향기', '출연영화': ['아이', '신과 함께-죄와 벌', '신과 함께-인과 연', '늑대소년', '증인']}
{'배우이름': '김준배', '출연영화': ['강릉', '악인은 너무 많다 2: 제주 실종사건의 전말', '부러진 화살', '바람의 언덕', '잉투기']}


#### find의 다양한 문법 (elemMatch)
* elemMatch 조건을 만족하는 값을 요소로 갖는 리스트를 검색

In [113]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re                   

connection = pymongo.MongoClient()
actor_db = connection.cine21
elemmatch_sample = actor_db.sample

In [115]:
elemmatch_sample.insert_many([
    {'results': [82, 85, 88]},
    {'results': [75, 88, 91]}
])

In [140]:
docs = elemmatch_sample.find({'results': {'$gte':75, '$lt':85}})

for doc in docs:
    print(doc)

{'_id': ObjectId('61d950ebf92a9ab9950228af'), 'results': [82, 85, 88]}
{'_id': ObjectId('61d950ebf92a9ab9950228b0'), 'results': [75, 88, 91]}


In [127]:
## 75 =< x =< 80 만족하는 x를 요소로 갖고 있는 리스트를 결과값으로 출력

docs = elemmatch_sample.find({'results': {'$elemMatch': {'$gte':75, '$lte':80}}})

for doc in docs:
    print(doc)

{'_id': ObjectId('61d950ebf92a9ab9950228b0'), 'results': [75, 88, 91]}


In [137]:
## 80 =< x =< 85 만족하는 x를 요소로 갖고 있는 리스트를 결과값으로 출력

docs = elemmatch_sample.find({'results': {'$elemMatch': {'$gte':80, '$lte':85}}})

for doc in docs:
    print(doc)

{'_id': ObjectId('61d950ebf92a9ab9950228af'), 'results': [82, 85, 88]}


## 실습

In [146]:
## 직업이 가수인 배우 중, 흥행지수가 가장 높은 배우순으로 10명을 출력

docs = actor.find({'직업':'가수'}, {'배우이름':1, '흥행지수':1, '_id':0}).sort('흥행지수', pymongo.DESCENDING).limit(10)

for doc in docs:
    print(doc)

{'배우이름': '임윤아', '흥행지수': 9812}
{'배우이름': '김소혜', '흥행지수': 1625}
{'배우이름': '서인국', '흥행지수': 1495}
{'배우이름': '찬열', '흥행지수': 1204}
{'배우이름': '한선화', '흥행지수': 1106}
{'배우이름': '송가인', '흥행지수': 1002}


In [159]:
## 모가디슈에 출연한 배우를 흥행지수가 높은 순으로 5명 출력

docs = actor.find({'출연영화': '모가디슈'}, {'배우이름':1 , '흥행지수':1, '_id':0}).sort('흥행지수', pymongo.DESCENDING).limit(5)

for doc in docs:
    print(doc)

{'배우이름': '김윤석', '흥행지수': 46285}
{'배우이름': '조인성', '흥행지수': 40500}
{'배우이름': '허준호', '흥행지수': 34683}
{'배우이름': '김소진', '흥행지수': 28917}
{'배우이름': '정만식', '흥행지수': 23125}
